# Import

In [1]:
import csv
import os
import sys

import numpy as np
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

## Add configuration file

In [2]:
sys.path.append("/home/jovyan/core/config/")
sys.path.append("/home/jovyan/core/util/")

In [3]:
from ALL import config 
from util import *

## Set condition

In [4]:
tqdm.pandas()
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)

In [5]:
s3 = S3Manager()

In [6]:
data_type = "20News"

In [7]:
transformer_model = "sentence-transformers/gtr-t5-large"

# Read data

In [8]:
df_path = s3.download(f"Preprocessing/{data_type}/master.csv")

In [9]:
df = pd.read_csv(df_path[0], index_col=0)

In [10]:
# sampling for test
# df.sample(n=1000, random_state=0)

In [11]:
labels_path = s3.download(f"Preprocessing/{data_type}/class.csv")

In [12]:
with open(labels_path[0], mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

In [13]:
max_model_num = config["vectorize"]["sentenceBERT"]["max_model_num"]

# Embedding

In [14]:
def get_sentenceBERT(texts,seed, path):
    model = SentenceTransformer(transformer_model)
    model.save(path)
    vectors = model.encode(texts)
    return vectors

In [ ]:
vectors_path = f"../../temporary/Vectorize/{data_type}/sentenceBERT/{transformer_model}/vector"
models_path = f"../../temporary/Vectorize/{data_type}/sentenceBERT/{transformer_model}/model"

for model_num in tqdm(range(max_model_num)):
    vectors = get_sentenceBERT(
        df.text.tolist(),
        seed=model_num,
        path=make_filepath(f"{models_path}/{model_num}"),
    )

    np.save(
        make_filepath(f"{vectors_path}/raw/{model_num}.npy"),
        np.stack(vectors),
    )


  0%|          | 0/1 [00:00<?, ?it/s]

## upload file

In [ ]:
s3.upload(
    f"../../temporary/Vectorize/{data_type}/sentenceBERT/{transformer_model}/", 
    f"Vectorize/{data_type}/sentenceBERT/{transformer_model}/"
)

In [ ]:
s3.delete_local_all()

In [ ]:
send_line_notify(f"end {data_type} sentenceBERT {transformer_model}")

In [ ]:
!free

In [ ]:
!df